<a href="https://colab.research.google.com/github/enricco000/MR4010.10/blob/main/Actividad_4_2_Clasificaci%C3%B3n_CNN_GTRSB_Equipo19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importación de librerías estándar
import os
import random
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
# Al igual que en la actividad anterior, el dataset está en google drive.
# Para correr este notebook, debes subir tu
# dataset a google drive también. En la entrega anterior, utilizamos el juego
# el siguiente juego de datos de Kaggle:
# https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign?resource=download
# Para poder accederlo desde este notebook
# necesitamos importar drive
from google.colab import drive
drive.mount('/content/gdrive')
# Definimos la ruta de la carpeta donde se encuentran las imágenes
PWD = '/content/gdrive/MyDrive/gtsrb'
# Definimos las rutas de las subcarpetas
TRAIN = os.path.join(PWD, 'Train')
TEST = os.path.join(PWD, 'Test')
META = os.path.join(PWD, 'Meta')

In [ ]:
# Importaciones para el preprocesamiento y la CNN
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Definición de los parámetros
num_classes = 43  # Tenemos 43 clases en este juego de datos
batch_size = 32
epochs = 30
img_height, img_width = 64, 64

# Generador de imágenes para el entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) 

# Generador de flujo de las imágenes para el entrenamiento, a partir del
# generador de imágenes
train_generator = train_datagen.flow_from_directory(
    directory=TRAIN,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training') # data de entrenamiento

# Generador de flujo de imágenes para validación
validation_generator = train_datagen.flow_from_directory(
    directory=TRAIN, # se usa el mismo directorio que en el entrenamiento
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation') # data de validación

# Definición del modelo. Adicional a lo que se hizo para la actividad pasada,
# en este caso vamos a añadir una capa de abandono después de cada capa de 
# agrupamiento. Dado que son varias clases, estamos tratando de encontrar
# las características más notorias para cada clase
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compilación del modelo
model.compile(optimizer=Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator, 
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs)
